### **Обработка данных**

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle

from matplotlib.ticker import FormatStrFormatter
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score 
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


*Функции*

In [2]:
# выводит статистику по заполненности набора данных
def print_stats(df):
    print(f"Количество полностью заполненных объектов: {len(df.dropna())}")
    print(f"Процент полностью заполненных объектов: {round(len(df.dropna()) / (len(df) / 100), 2)}")

In [3]:
# выводит процент пропущенных значений
def print_missing_values(df):
    percent_missing = (df.isna().sum() / (len(df) / 100)).sort_values(ascending=False)
    print(f"Процент пропущенных значений:\n{percent_missing}")

In [4]:
# выводит уникальные и часто встречаемые значения для группы колонок
def print_uniq_and_top(df, filled_column, bind_column):
    print("Уникальные значения ", bind_column, " для строк с заполненным ", filled_column, ":", df[bind_column].unique())
    print("Самое часто встречаемое значение ", bind_column, " для строк с заполненным ", filled_column, ":", df[bind_column].describe()['top'])

#### *Data Preparation*

In [5]:
df_out_pkl = 0
with open("data/ga_sessions.pkl", 'rb') as f:
    df_out_pkl = pickle.load(f)
df_out_pkl.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [6]:
print_stats(df_out_pkl)

Количество полностью заполненных объектов: 14940
Процент полностью заполненных объектов: 0.8


In [7]:
print_missing_values(df_out_pkl)

Процент пропущенных значений:
device_model                99.121633
utm_keyword                 58.174009
device_os                   57.533002
utm_adcontent               18.043410
utm_campaign                11.806346
device_brand                 6.380394
utm_source                   0.005215
geo_country                  0.000000
device_browser               0.000000
device_screen_resolution     0.000000
session_id                   0.000000
device_category              0.000000
client_id                    0.000000
utm_medium                   0.000000
visit_number                 0.000000
visit_time                   0.000000
visit_date                   0.000000
geo_city                     0.000000
dtype: float64


##### Обработка пропущенных значений device_model

In [8]:
df_out_pkl['device_model'].describe()

count                    16338
unique                     104
top       AuMdmADEIoPXiWpTsBEj
freq                      9778
Name: device_model, dtype: object

In [9]:
top_device_model = df_out_pkl['device_model'].describe()['top']

Вывод пустых значений device_model

In [10]:
df_out_pkl[df_out_pkl['device_model'].isna()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,None,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


Выводим заполненные значения

In [11]:
df_dm_fill = df_out_pkl[~(df_out_pkl['device_model'].isna())]
df_dm_fill

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
62,9055656589238457290.1635504877.1635504877,2108434352.1635505098,2021-10-29,13:54:37,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Meizu,qBRdfuuhOnnqwSqNiPOv,360x744,Chrome,Russia,Moscow
65,905565998839028208.1635310063.1635310063,210843514.1635310064,2021-10-27,07:47:43,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,OnePlus,AshpvdJdReafUzEMmWGr,384x832,Android Webview,Russia,Saint Petersburg
66,905565998839028208.1636951969.1636951969,210843514.1635310064,2021-11-15,07:52:49,2,ISrKoXQCxqqYvAZICvjs,smm,NaN,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,OnePlus,AshpvdJdReafUzEMmWGr,384x832,Android Webview,Russia,Saint Petersburg
165,9056202067269505745.1640092368.1640092368,2108561356.1640092369,2021-12-21,16:12:48,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,384x854,Chrome,Russia,Moscow
208,9056422519349747445.1638964982.1638964982,2108612684.1638964981,2021-12-08,15:03:02,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,RrhnkuoaqckNtJpAZDzH,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,320x640,Chrome,Russia,Saint Petersburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859362,9052676375860711226.1639943995.1639943995,2107740467.1639943994,2021-12-19,22:59:55,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Izhevsk
1859404,9052904464392788371.1638799763.1638799763,2107793573.1638799763,2021-12-06,17:09:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Nokia,aCLaTVQlHcXbUnQYQvIg,412x892,Chrome,Russia,Novosibirsk
1859535,9053425229175288131.1636659526.1636659526,2107914823.1636659523,2021-11-11,22:38:46,1,faqsogjxCvbseFqupueU,banner,dZqEgyoxhtbeLFMtnnVR,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,385x769,Chrome,Russia,Samara
1859612,9053730012943175107.1640761072.1640761072,2107985786.1640320451,2021-12-29,09:57:52,2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x760,Chrome,Russia,Vladivostok


In [12]:
print_uniq_and_top(df_dm_fill, 'device_model', 'device_category')
print_uniq_and_top(df_dm_fill, 'device_model', 'device_brand')
print_uniq_and_top(df_dm_fill, 'device_model', 'device_os')
print_uniq_and_top(df_dm_fill, 'device_model', 'device_browser')

Уникальные значения  device_category  для строк с заполненным  device_model : ['mobile' 'tablet' 'desktop']
Самое часто встречаемое значение  device_category  для строк с заполненным  device_model : mobile
Уникальные значения  device_brand  для строк с заполненным  device_model : ['Meizu' 'OnePlus' '(not set)' 'Vivo' 'Nokia' 'Motorola' 'Xiaomi' 'Inoi'
 'ZTE' 'Oukitel' 'HTC' 'Symphony' 'BQ' 'TurboPad' 'Kata' 'Nuu' 'Advan'
 'Coolpad' 'Dynamic']
Самое часто встречаемое значение  device_brand  для строк с заполненным  device_model : (not set)
Уникальные значения  device_os  для строк с заполненным  device_model : ['Android' 'iOS' '(not set)' 'Tizen' 'Windows Phone' 'Nokia']
Самое часто встречаемое значение  device_os  для строк с заполненным  device_model : Android
Уникальные значения  device_browser  для строк с заполненным  device_model : ['Chrome' 'Android Webview' 'Opera' 'Firefox' 'YaBrowser'
 'Samsung Internet' 'Mozilla' 'Safari' 'Opera Mini' 'UC Browser'
 'Internet Explorer' 'Nokia5

Когда бренд не определяется, ему ставится значение модели AuMdmADEIoPXiWpTsBEj, которое соответствует любому устройству, в том числе мобильному, поэтому заполним им пустые значения device_model

In [13]:
df_out_pkl['device_model'] = df_out_pkl['device_model'].fillna(top_device_model)

In [14]:
print_missing_values(df_out_pkl)

Процент пропущенных значений:
utm_keyword                 58.174009
device_os                   57.533002
utm_adcontent               18.043410
utm_campaign                11.806346
device_brand                 6.380394
utm_source                   0.005215
geo_country                  0.000000
device_browser               0.000000
device_screen_resolution     0.000000
device_model                 0.000000
session_id                   0.000000
device_category              0.000000
client_id                    0.000000
utm_medium                   0.000000
visit_number                 0.000000
visit_time                   0.000000
visit_date                   0.000000
geo_city                     0.000000
dtype: float64


##### Обработка пропущенных значений utm_keyword

In [15]:
df_out_pkl['utm_keyword'].describe()

count                   777981
unique                    1219
top       puhZPIYqKXeFPaUviSjo
freq                    506819
Name: utm_keyword, dtype: object

Выводим заполненные значения

In [16]:
top_key_word = df_out_pkl['utm_keyword'].describe()['top']
df_ukw_fill = df_out_pkl[~(df_out_pkl['utm_keyword'].isna())]
df_ukw_fill 

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk
6,9055455318486370642.1640843788.1640843788,2108387490.1640843602,2021-12-30,08:56:28,1,TxKUcPpthBDPieTGmVhx,cpc,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,PwscUHjoUJDrtfWESIHj,tablet,Android,Lenovo,AuMdmADEIoPXiWpTsBEj,602x1029,YaBrowser,Russia,Saint Petersburg
8,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860036,9055401700113249881.1639446112.1639446112,2108375006.1639446105,2021-12-14,04:41:52,1,ZpYIoDJMcFzVoPFsHGJL,banner,TmThBvoCcwkCZZUWACYq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,ZTE,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Blagoveshchensk
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,None,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk


Вывод пустых значений

In [17]:
df_out_pkl[df_out_pkl['utm_keyword'].isna()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
5,9055447192389856083.1622453074.1622453074,2108385598.1622453075,2021-05-31,12:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Apple,AuMdmADEIoPXiWpTsBEj,375x812,Safari,Russia,Saint Petersburg
7,9055461992850812764.1626107740.1626107740,2108389044.1626107740,2021-07-12,19:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Saint Petersburg
9,9055466554104774132.1624800757.1624800757,2108390106.1624800756,2021-06-27,16:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860030,9055382948278467242.1631877802.1631877802,2108370640.1631877802,2021-09-17,14:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Saint Petersburg
1860032,9055394269810294140.1629912447.1629912447,2108373276.1629912444,2021-08-25,20:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x800,Android Webview,Russia,Saint Petersburg
1860034,9055397194683347295.1630237022.1630237022,2108373957.1630237023,2021-08-29,14:00:00,1,ISrKoXQCxqqYvAZICvjs,blogger_stories,zfwIehuEfWYdYrEZgRLo,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Apple,AuMdmADEIoPXiWpTsBEj,414x896,Safari,Russia,Zheleznodorozhny
1860035,9055398929844789828.1624891784.1624891784,2108374361.1624891972,2021-06-28,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,vXsFkagGabkcWKlgLzSg,NaN,NaN,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,320x676,Chrome,Russia,Naro-Fominsk


In [18]:
print_uniq_and_top(df_ukw_fill, 'utm_keyword', 'utm_adcontent')

Уникальные значения  utm_adcontent  для строк с заполненным  utm_keyword : ['vCIpmpaGBnIQhyYNkXqp' 'xhoenQgDQsgfEPYNPwKO' 'LcGIUNPUAmXtQJaDfFBR'
 'JNHcPlZPxEMWDnRiyoBf' nan 'LLfCasrxQzJIyuldcuWy' 'DZYjhfIUfdqhfuTNUmjn'
 'FkiRXDLOWtzVfvhEkhNo' 'xnyHaukLtAvgViiZSyBC' 'ESUnXCsdWADovskBLvBO'
 'SOkCdPxfUcZUzzOdgGES' 'yYdBRbPmBMUZHXwqGxNx' 'lBvcHmZkKpwCXdgWSQYO'
 'AdeErYgVTbRcAWtHrMHq' 'PkybGvWbaqORmxjNunqZ' 'SAVVWaMghGnnvPOqMOIt'
 'lbhUYwMzoYJqJaUaTDba' 'LxluDbGsLnaemhTtGuvB' 'OJiWyBKOyDITzXCZRSMH'
 'qhEmhjPXvwgEHdBikgEQ' 'XSkXBCPfnJjvxbfeewtd' 'EteMoEECGsaJeMnuvAZD'
 'YTDFqIabKsQVGozQYoPf' 'fxKLUhFToKQtGIyvjZXQ' 'dUuXlWzvmhDSyclWRhNP'
 'IwBedorwDIzxDRIZUTNo' 'lXYxbSFluucyYXDQeIHX' 'LBoFGHDbSeBOgvTnNlmS'
 'DaehHXyBdjcdSRnPiAQn' 'nNqUcgFgcqQbTVSvgaHr' 'ptYJgYxtYQLiZcgpjmXe'
 'aYAcKhelKzYpXrRYknSP' 'WvCMRUAbanZQkQfWaSEa' 'sMBIidTLSrYkjsCEvwht'
 'ailGqjxuJsYcAqCUFSXX' 'qMvKVRkKgfEwmAJomJUe' 'TuyPWsGQruPMpKvRxeBF'
 'FNSBWzZNPqRWMLiASvpg' 'WbmadpYgwtVCcAwGJoXb' 'IyvBPOpVqcFCBjbgvbvx'
 'xKBQVMarl

In [19]:
print_uniq_and_top(df_ukw_fill, 'utm_keyword', 'utm_campaign')

Уникальные значения  utm_campaign  для строк с заполненным  utm_keyword : ['LEoPHuyFvzoNfnzGgfcd' 'FTjNLDyTrXaWYgZymFkV' 'LTuZkdKfxRGVceoWkVyg'
 'gecBYcKZCPMcVYdSSzKP' nan 'LwJZxKuWmvOhPsaCeRjG' 'okTXSMadDkjvntEHzIjp'
 'zxoiLxhuSIFrCeTLQVWZ' 'TmThBvoCcwkCZZUWACYq' 'foFTSdUvNqqkPzZvgiqt'
 'bxOTvPtyGSdUrbwoXCPO' 'XHNUiSKKGTEpSAlaVMFQ' 'dZqEgyoxhtbeLFMtnnVR'
 'UvuMsOSDBWQGOIbDbXfV' 'sbJRYgVfvcnqKJNDDYIr' 'XGYOaJEasWTwAKNdCGVX'
 'QdLfySaGXolfTBSNVfHn' 'RoDitORHdzGfGhNCyEMy' 'JajANoFxoqXfKRNBUhzx'
 'UEtHtwAEXfprDUERwqqj' 'ascPqxFuFewWWZSVMpkh' 'SgIUDYUKnyWHVowUOqid'
 'xGRbRuBkKsQPkKARPeOn' 'emlBaVkgLVRqbEVOSSXt' 'UjApcvnaHtkydRkrLYuv'
 'kVOrIKZFrEYGvixPclal' 'jqlUOdZBNZYfInQVcZlS' 'LUlvACDKkkOkiSuiwaBs'
 'CdkIkBrvEVomSgvHzYvj' 'vFcAhRxLfOWKhvxjELkx' 'yCnTwlCLkqZHwAglrEll'
 'bJJuEXRheRIxXEaYIXqM' 'lDZWtjMawBaqetnVFboy' 'bgTYkDHjOsJzMUtoGhiQ'
 'hkvDVxqLOzGjGaoNiNzN' 'nSReTmyFtbSjlPrTKoaX' 'DZlFqIVHUBIDaQoarvIZ'
 'IKQsApKuPmZqqmhieEgf' 'WlbWUObZWvsimzdFdLYw' 'kwdmElMUPDZaLQdgjcsI'
 'PTQlxxEuqj

In [20]:
print_uniq_and_top(df_ukw_fill, 'utm_keyword', 'utm_medium')

Уникальные значения  utm_medium  для строк с заполненным  utm_keyword : ['banner' 'cpm' 'cpc' 'referral' '(none)' 'organic' 'smm'
 'blogger_channel' 'email' 'app' 'vk_smm' 'push' 'partner' 'smartbanner'
 'info_text' 'outlook' 'clicks' 'landing' 'blogger_stories' 'post' 'tg'
 'qr' 'cpa' '(not set)' 'fb_smm' 'nkp' 'google_cpc' 'blogger_header'
 'users_msk' 'cpv' 'ok_smm' 'Sbol_catalog' 'sms' 'landing_interests'
 'yandex_cpc' 'web_polka' 'static' 'linktest' 'CPM' 'medium' 'promo_sbol'
 'desktop' 'dom_click' 'main_polka' 'link' 'stories' 'article'
 'promo_sber']
Самое часто встречаемое значение  utm_medium  для строк с заполненным  utm_keyword : banner


In [21]:
print_uniq_and_top(df_ukw_fill, 'utm_keyword', 'utm_source')

Уникальные значения  utm_source  для строк с заполненным  utm_keyword : ['ZpYIoDJMcFzVoPFsHGJL' 'MvfHsxITijuriZxsqZqt' 'TxKUcPpthBDPieTGmVhx'
 'gVRrcxiDQubJiljoTbGm' 'BHcvLfOaCWvWTykYqHVe' 'fDLlAcSmythWSCVMvqvL'
 'kjsLglQLzykiRbcDiGcD' 'vFcAhRxLfOWKhvxjELkx' 'ISrKoXQCxqqYvAZICvjs'
 'IZEXUFLARCUMynmHNBGo' 'jaSOmLICuBzCFqHfBdRg' 'bByPQxmDaMXgpHeypKSM'
 'nSReTmyFtbSjlPrTKoaX' 'dGlVSdmIlgWDyOPjfwwy' 'NGNkCWwKgYFmiCCeZVxg'
 'GpAkIXsclxDGyILfNlrR' 'oZCzWSykfixnjMPDNjSU' 'dyicZQGoeASogoSafjEh'
 'fgymSoTvjKPEgaIJqsiH' 'QxAxdyPLuQMEcrdZWdWb' 'eLzNJHzPelJpEyBwMrKo'
 'aXQzDWsJuGXeBXexNHjc' 'nmfptFmSirEqNzAzqbXA' 'RmEBuqrriAfAVsLQQmhk'
 'cAqxcRdSSFAyCPUxQHqy' 'iNFgfQPqHPBuvGCYtrQE' 'HFaOtpcChAlcMuxEAlpu'
 'eimRuUrNhZLAYcwRrNXu' 'YlsczTIyBSwTLNtuDkCd' 'KgicpPxiEQfzPlPwQZJq'
 'DnEUulZAecfGPvdtZBYS' 'GmILPdZyuAVJCPsUBHeN' 'XiUifkjKLLnomcDRhswp'
 'YclHumxPxSxgzHfvCaeF' 'oCqKpnSZJeYOVZTgTmKR' 'ngkgBNjlzLYBofkljaBo'
 'faqsogjxCvbseFqupueU' 'nrKihqcWGIzDsOqljdAv' 'xEbgdGZJlqXAaRmeJQdW'
 'LlBOVIARRTjfgnQN

In [22]:
df_out_pkl[(df_out_pkl['utm_medium'] != 'banner')]['utm_keyword'].describe()

count                   457403
unique                    1219
top       puhZPIYqKXeFPaUviSjo
freq                    186241
Name: utm_keyword, dtype: object

In [23]:
df_out_pkl[(df_out_pkl['utm_source'] != 'ZpYIoDJMcFzVoPFsHGJL')]['utm_keyword'].describe()

count                   430706
unique                    1219
top       puhZPIYqKXeFPaUviSjo
freq                    159544
Name: utm_keyword, dtype: object

In [24]:
df_out_pkl[(df_out_pkl['utm_campaign'] != 'LEoPHuyFvzoNfnzGgfcd')]['utm_keyword'].describe()

count                   546808
unique                    1219
top       puhZPIYqKXeFPaUviSjo
freq                    275646
Name: utm_keyword, dtype: object

Таким образом, от связанных колонок самое часто встречаемое значение не зависит, что позволяет заполнить им пустые значения utm_keyword

In [25]:
df_out_pkl['utm_keyword'] = df_out_pkl['utm_keyword'].fillna(top_key_word)

In [26]:
print_missing_values(df_out_pkl)

Процент пропущенных значений:
device_os                   57.533002
utm_adcontent               18.043410
utm_campaign                11.806346
device_brand                 6.380394
utm_source                   0.005215
session_id                   0.000000
geo_country                  0.000000
device_browser               0.000000
device_screen_resolution     0.000000
device_model                 0.000000
utm_keyword                  0.000000
device_category              0.000000
client_id                    0.000000
utm_medium                   0.000000
visit_number                 0.000000
visit_time                   0.000000
visit_date                   0.000000
geo_city                     0.000000
dtype: float64


##### Обработка пропущенных значений device_os

In [27]:
df_out_pkl['device_os'].describe()

count      789904
unique         13
top       Android
freq       464054
Name: device_os, dtype: object

In [28]:
top_device_os_m = df_out_pkl.loc[(df_out_pkl['device_category'] == 'mobile'), 'device_os'].describe()['top']
top_device_os_t = df_out_pkl.loc[(df_out_pkl['device_category'] == 'tablet'), 'device_os'].describe()['top']
top_device_os_d = df_out_pkl.loc[(df_out_pkl['device_category'] == 'desktop'), 'device_os'].describe()['top']

In [29]:
df_do_fill = df_out_pkl[~(df_out_pkl['device_os'].isna())]
df_do_fill 

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk
6,9055455318486370642.1640843788.1640843788,2108387490.1640843602,2021-12-30,08:56:28,1,TxKUcPpthBDPieTGmVhx,cpc,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,PwscUHjoUJDrtfWESIHj,tablet,Android,Lenovo,AuMdmADEIoPXiWpTsBEj,602x1029,YaBrowser,Russia,Saint Petersburg
8,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860036,9055401700113249881.1639446112.1639446112,2108375006.1639446105,2021-12-14,04:41:52,1,ZpYIoDJMcFzVoPFsHGJL,banner,TmThBvoCcwkCZZUWACYq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,ZTE,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Blagoveshchensk
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,None,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk


In [30]:
df_out_pkl[df_out_pkl['device_os'].isna()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,puhZPIYqKXeFPaUviSjo,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,puhZPIYqKXeFPaUviSjo,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
5,9055447192389856083.1622453074.1622453074,2108385598.1622453075,2021-05-31,12:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,Apple,AuMdmADEIoPXiWpTsBEj,375x812,Safari,Russia,Saint Petersburg
7,9055461992850812764.1626107740.1626107740,2108389044.1626107740,2021-07-12,19:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Saint Petersburg
9,9055466554104774132.1624800757.1624800757,2108390106.1624800756,2021-06-27,16:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860030,9055382948278467242.1631877802.1631877802,2108370640.1631877802,2021-09-17,14:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,puhZPIYqKXeFPaUviSjo,mobile,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Saint Petersburg
1860032,9055394269810294140.1629912447.1629912447,2108373276.1629912444,2021-08-25,20:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x800,Android Webview,Russia,Saint Petersburg
1860034,9055397194683347295.1630237022.1630237022,2108373957.1630237023,2021-08-29,14:00:00,1,ISrKoXQCxqqYvAZICvjs,blogger_stories,zfwIehuEfWYdYrEZgRLo,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,Apple,AuMdmADEIoPXiWpTsBEj,414x896,Safari,Russia,Zheleznodorozhny
1860035,9055398929844789828.1624891784.1624891784,2108374361.1624891972,2021-06-28,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,vXsFkagGabkcWKlgLzSg,NaN,puhZPIYqKXeFPaUviSjo,mobile,None,Samsung,AuMdmADEIoPXiWpTsBEj,320x676,Chrome,Russia,Naro-Fominsk


Во многих случаях случаях device_os зависит от device_brand, и в этих записях мы точно можем заполнить отсутствующие значения

In [31]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'Apple'), 'device_os'].describe()

count     207098
unique         1
top          iOS
freq      207098
Name: device_os, dtype: object

In [32]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'Apple'), 'device_os'].unique()

array([None, 'iOS'], dtype=object)

In [33]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'Apple'), 'device_category'].unique()

array(['mobile', 'tablet', 'desktop'], dtype=object)

In [34]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'Apple'), 'device_category'].describe()

count     551088
unique         3
top       mobile
freq      544793
Name: device_category, dtype: object

In [35]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == 'Apple') & ((df_out_pkl['device_category'] == 'mobile') | (df_out_pkl['device_category'] == 'tablet')) & (df_out_pkl['device_os'].isna())), 'device_os']  = 'iOS'

In [36]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'BlackBerry'), 'device_os'].unique()

array([None, 'Android', 'BlackBerry'], dtype=object)

In [37]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == 'BlackBerry') & (df_out_pkl['device_os'] == 'BlackBerry'))].shape

(27, 18)

In [38]:
df_out_pkl.loc[(df_out_pkl['device_brand'] == 'BlackBerry'), 'device_os'].describe()

count          92
unique          2
top       Android
freq           65
Name: device_os, dtype: object

BlackBerry OS - достаточно редкая в текущее время система, и при анализе она не окажет существенного влияния на результат, поэтому заполняем device_os для этого бренда значением самой часто встречаемой ОС  

In [39]:
df_out_pkl.loc[(df_out_pkl['device_brand'] != 'Apple'), 'device_os'].unique()

array(['Android', None, 'Windows', 'Linux', 'Macintosh', '(not set)',
       'Chrome OS', 'iOS', 'BlackBerry', 'Tizen', 'Firefox OS', 'Samsung',
       'Windows Phone', 'Nokia'], dtype=object)

In [40]:
df_do_fill.loc[(df_do_fill['device_brand'] != 'Apple'), 'device_os'].unique()

array(['Android', 'Windows', 'Linux', 'Macintosh', '(not set)',
       'Chrome OS', 'iOS', 'BlackBerry', 'Tizen', 'Firefox OS', 'Samsung',
       'Windows Phone', 'Nokia'], dtype=object)

In [43]:
print(f"Самые часто встречаемые мобильные ОС: {top_device_os_m} \nПланшетные ОС: {top_device_os_t}")

Самые часто встречаемые мобильные ОС: Android 
Планшетные ОС: Android


In [44]:
list_brands = df_do_fill.loc[(((df_do_fill['device_os'] == top_device_os_m) | (df_do_fill['device_os'] == top_device_os_t)) & ((df_do_fill['device_category'] == 'mobile') | (df_do_fill['device_category'] == 'tablet'))), 'device_brand'].unique()
list_brands

array(['Huawei', 'Samsung', 'Lenovo', 'Xiaomi', 'Meizu', 'OnePlus',
       'Realme', 'OPPO', '(not set)', 'Philips', 'Vivo', 'Nokia',
       'Alcatel', 'LG', 'BQ', 'Tecno', 'Asus', 'itel', 'Infinix', 'ZTE',
       'Wiko', 'Google', 'Sony', 'Wileyfox', 'Blackview', 'Cubot',
       'DOOGEE', 'DEXP', 'Motorola', 'TP-Link', 'Hisense', 'Acer',
       'Oukitel', 'LeEco', 'Prestigio', 'POCO', 'Vsmart', 'HTC',
       'Ulefone', 'CAT', 'Leagoo', 'InFocus', 'Inoi', 'BlackBerry',
       'Micromax', 'Umidigi', 'Sharp', 'Jiake', 'ZOJI', 'Yuntab',
       'Mozilla', 'Neffos', 'Highscreen', 'Karbonn', 'TCL', 'BLU',
       'Haier', 'Vertex', 'Coolpad', 'HOMTOM', 'LeTV', 'A1',
       'General Mobile', 'Gome', 'Egreat', 'Mito', 'SenseIT', 'Archos',
       'Keecoo', 'Vernee', 'Panasonic', 'InnJoo', 'Iris', 'Black Fox',
       'Lava', 'myPhone', 'Nomu', 'AGM', 'Nuu', 'UGOOS', 'Alldocube',
       'MTC', 'Komu', 'Qbex', 'Symphony', 'Wigor', 'Oysters', 'Fly',
       'Gionee', 'Artel', 'Ananda', 'Smartisan', '

In [45]:
df_out_pkl.loc[((df_out_pkl['device_brand'].isin(list(list_brands))) & (df_out_pkl['device_category'] == 'mobile') & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_m
df_out_pkl.loc[((df_out_pkl['device_brand'].isin(list(list_brands))) & (df_out_pkl['device_category'] == 'tablet') & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_t

Теперь проверяем характеристики device_browser и device_category

In [46]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_browser'].unique()

array(['YaBrowser', 'Chrome', 'Safari', 'Firefox', 'Opera', 'Edge',
       'Instagram 208.0.0.32.135 Android',
       'Instagram 209.0.0.21.119 Android', '(not set)',
       'Mozilla Compatible Agent', 'Coc Coc', 'Samsung Internet',
       'Android', '[FBAN', 'Puffin', 'Internet Explorer', 'MRCHROME',
       'Instagram 199.1.0.34.119 Android', 'UC Browser', 'SeaMonkey',
       'Instagram 194.0.0.36.172 Android',
       'Instagram 202.0.0.37.123 Android', 'Mozilla',
       'Instagram 192.0.0.35.123 Android', 'Maxthon', 'Android Webview',
       'Instagram 158.0.0.30.123 Android', 'Konqueror'], dtype=object)

Встроенные браузеры так же позволяют однозначно определить систему

In [47]:
df_out_pkl.loc[((df_out_pkl['device_browser'] == 'Safari') & (df_out_pkl['device_os'].isna())), 'device_os'] = 'Macintosh'

In [48]:
df_out_pkl.loc[((df_out_pkl['device_os'].isna()) & (df_out_pkl['device_browser'] == 'Samsung Internet')), 'device_os'] = 'Android'

In [49]:
df_out_pkl.loc[((df_out_pkl['device_os'].isna()) & ((df_out_pkl['device_browser'] == 'Edge') | (df_out_pkl['device_browser'] == 'Internet Explorer'))), 'device_os'] = 'Windows'

In [50]:
df_out_pkl.loc[((df_out_pkl['device_os'].isna()) & (df_out_pkl['device_browser'].str.contains('Android', na=False))), 'device_os'] = 'Android' 

In [51]:
df_out_pkl.loc[(df_out_pkl['device_browser'] == 'Mozilla Compatible Agent'), 'device_category'].unique()

array(['desktop'], dtype=object)

In [52]:
df_out_pkl.loc[((df_out_pkl['device_browser'] == 'Mozilla Compatible Agent') & (df_out_pkl['device_os'].isna())), 'device_category'].unique()

array(['desktop'], dtype=object)

In [53]:
df_out_pkl.loc[(df_out_pkl['device_browser'] == 'Mozilla Compatible Agent'), 'device_os'].describe()

count           302
unique            3
top       (not set)
freq            260
Name: device_os, dtype: object

In [54]:
df_out_pkl.loc[(df_out_pkl['device_browser'] == 'Mozilla Compatible Agent'), 'device_brand'].describe()

count     60
unique     1
top         
freq      60
Name: device_brand, dtype: object

In [55]:
df_out_pkl.loc[(df_out_pkl['device_browser'] == 'Mozilla Compatible Agent')]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
8519,9093064959820033783.1637871353.1637871353,2117144167.1637871351,2021-11-25,23:15:53,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,(not set),None,AuMdmADEIoPXiWpTsBEj,800x600,Mozilla Compatible Agent,Iran,(not set)
10828,9102899502911587345.1629488145.1629488145,2119433950.1629488145,2021-08-20,22:00:00,1,nSReTmyFtbSjlPrTKoaX,banner,BVKxkCOHKUOvkpbrLMgZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,None,,AuMdmADEIoPXiWpTsBEj,768x1024,Mozilla Compatible Agent,Russia,Saint Petersburg
24764,9164669134152695542.1630043894.1630043894,2133815813.1630043894,2021-08-27,08:00:00,1,nSReTmyFtbSjlPrTKoaX,banner,BVKxkCOHKUOvkpbrLMgZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,None,,AuMdmADEIoPXiWpTsBEj,768x1024,Mozilla Compatible Agent,Russia,Moscow
29900,9187225035383329340.1637438015.1637438015,2139067518.1637438012,2021-11-20,22:53:35,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,(not set),None,AuMdmADEIoPXiWpTsBEj,800x600,Mozilla Compatible Agent,Russia,Nizhny Tagil
31090,9192716748074205719.1639924247.1639924247,2140346157.1639924247,2021-12-19,17:30:47,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,(not set),None,AuMdmADEIoPXiWpTsBEj,800x600,Mozilla Compatible Agent,Russia,(not set)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858913,905062508407902236.1630359614.1630359614,210726286.1630359580,2021-08-31,00:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,,AuMdmADEIoPXiWpTsBEj,1920x1080,Mozilla Compatible Agent,Russia,Saint Petersburg
1858914,905062508407902236.1630359673.1630359673,210726286.1630359580,2021-08-31,00:00:00,3,kjsLglQLzykiRbcDiGcD,cpc,bJJuEXRheRIxXEaYIXqM,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,,AuMdmADEIoPXiWpTsBEj,1920x1080,Mozilla Compatible Agent,Russia,Saint Petersburg
1858915,905062508407902236.1630359674.1630359674,210726286.1630359580,2021-08-31,00:00:00,4,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,,AuMdmADEIoPXiWpTsBEj,1920x1080,Mozilla Compatible Agent,Russia,Saint Petersburg
1859651,9053936214320221766.1637486150.1637486167,2108033796.1637486150,2021-11-21,12:16:07,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,puhZPIYqKXeFPaUviSjo,desktop,Linux,None,AuMdmADEIoPXiWpTsBEj,1600x900,Mozilla Compatible Agent,Russia,Moscow


Mozilla Compatible Agent определяется как браузер в данных траффика в случаях, когда реальные люди заходят через VPN, когда заходят боты или происходит ошибка в whois, к тому же, этих данных немного, поэтому их можно удалить

In [56]:
df_out_pkl = df_out_pkl.loc[~(df_out_pkl['device_browser'] == 'Mozilla Compatible Agent')]

FBAN - это встроенный в приложение Facebook браузер на iOS, поэтому его мы тоже можем определить

In [59]:
df_out_pkl.loc[((df_out_pkl['device_os'].isna()) & (df_out_pkl['device_browser'] == '[FBAN')), 'device_os']  = 'iOS'

Посмотрим, какие ещё строки у нас остались с незаполненным device_os

In [60]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_browser'].unique()

array(['YaBrowser', 'Chrome', 'Firefox', 'Opera', '(not set)', 'Coc Coc',
       'Puffin', 'MRCHROME', 'UC Browser', 'SeaMonkey', 'Mozilla',
       'Maxthon', 'Konqueror'], dtype=object)

In [63]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_browser'].describe()

count     210518
unique        13
top       Chrome
freq      147684
Name: device_browser, dtype: object

In [65]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].unique()

array(['desktop', 'mobile', 'tablet'], dtype=object)

In [66]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].describe()

count      210518
unique          3
top       desktop
freq       210239
Name: device_category, dtype: object

Рассмотрим категорию desktop

In [67]:
df_out_pkl.loc[(df_out_pkl['device_category'] == 'desktop'), 'device_os'].describe()

count      156262
unique          8
top       Windows
freq        98121
Name: device_os, dtype: object

In [68]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & (df_out_pkl['device_os'].isna())), 'device_brand'].unique()

array(['', 'Xiaomi', 'Huawei', 'Samsung', 'Nokia', 'Asus', 'Beelink',
       'OPPO', 'Apple', '(not set)', 'OnePlus', 'Philips', 'Realme'],
      dtype=object)

Очевидно, что декстопный компьютер бренда Apple управляется ОС Macintosh

In [69]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & (df_out_pkl['device_os'].isna()) & (df_out_pkl['device_brand'] == 'Apple')), 'device_os'] = 'Macintosh'

Взглянем на оставшиеся после этих действий бренды, категории и браузеры

In [89]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & (df_out_pkl['device_os'].isna())), 'device_brand'].unique()

array(['', 'Xiaomi', 'Huawei', 'Samsung', 'Nokia', 'Asus', 'Beelink',
       'OPPO', '(not set)', 'OnePlus', 'Philips', 'Realme'], dtype=object)

In [92]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & (df_out_pkl['device_os'].isna())), 'device_brand'].describe()

count     210236
unique        12
top             
freq      210195
Name: device_brand, dtype: object

In [93]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == '') & (df_out_pkl['device_os'].isna())), 'device_browser'].unique()

array(['YaBrowser', 'Chrome', 'Firefox', 'Opera', '(not set)', 'Coc Coc',
       'Puffin', 'MRCHROME', 'UC Browser', 'SeaMonkey', 'Mozilla',
       'Maxthon', 'Konqueror'], dtype=object)

In [94]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == '') & (df_out_pkl['device_os'].isna())), 'device_browser'].describe()

count     210407
unique        13
top       Chrome
freq      147585
Name: device_browser, dtype: object

In [96]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == '(not set)') & (df_out_pkl['device_os'].isna())), 'device_browser'].unique()

array(['Chrome'], dtype=object)

In [97]:
df_out_pkl.loc[((df_out_pkl['device_brand'] == '(not set)') & (df_out_pkl['device_os'].isna())), 'device_category'].unique()

array(['desktop'], dtype=object)

Очевидно, что не определённые бренды в категории desktop - это обычные ПК, поэтому заполним их device_os самой часто встречаемой ОС

In [98]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & ((df_out_pkl['device_brand'] == '') | (df_out_pkl['device_brand'] == '(not set)')) & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_d

In [99]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_brand'].unique()

array(['China Phone', '', 'Flylion', 'Itoos', 'Walton', 'Xiaomi',
       'Xiaolajiao', 'Condor', 'Razer', 'Fujitsu', 'PPTV', 'Huawei',
       'Samsung', 'RCA', 'Cube', 'AT&T', 'Tonbux', 'Nokia', 'T-Mobile',
       'Smartfren', 'KingSing', 'Dragon Touch', 'Fero', 'Land Rover',
       'Tanix', 'Asus', 'Mlais', 'Beelink', 'Orbic', 'OPPO', 'Honeywell',
       'RED', 'Sonim', 'Jiayu', 'Leegoog', 'Star', 'Ellipsis', 'LTC',
       'Motive', 'OnePlus', 'M-HORSE', 'Winnovo', 'Nomi', 'Philips',
       'Wings Mobile', 'How', 'Maze', 'Realme', 'Tagital', 'Maxvi'],
      dtype=object)

In [102]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_brand'].describe()

count     318
unique     50
top          
freq      212
Name: device_brand, dtype: object

In [100]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].unique()

array(['mobile', 'tablet', 'desktop'], dtype=object)

In [103]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].describe()

count        318
unique         3
top       mobile
freq         248
Name: device_category, dtype: object

In [104]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_browser'].unique()

array(['Chrome', 'Opera', 'YaBrowser', 'Firefox', 'UC Browser'],
      dtype=object)

Оставшиеся мобильные устройства, чьи бренды известны, работают под управлением ОС семейства Android, поэтому заполняем их device_os следующим образом

In [105]:
df_out_pkl.loc[(((df_out_pkl['device_category'] == 'mobile') | (df_out_pkl['device_category'] == 'tablet')) & (df_out_pkl['device_brand'] != '') & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_m

In [106]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_brand'].unique()

array(['', 'Xiaomi', 'Huawei', 'Samsung', 'Nokia', 'Asus', 'Beelink',
       'OPPO', 'OnePlus', 'Philips', 'Realme'], dtype=object)

In [108]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].unique()

array(['mobile', 'desktop', 'tablet'], dtype=object)

In [109]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()), 'device_category'].describe()

count        251
unique         3
top       mobile
freq         197
Name: device_category, dtype: object

In [110]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna())]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
24009,9161436681631382037.1625242134.1625242134,2133063199.1625242133,2021-07-02,19:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,JwYIveaHVpeeRZloQCfF,NaN,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,1440x900,Chrome,Russia,Moscow
26043,9170117347413292512.1631024608.1631024608,2135084324.1631024608,2021-09-07,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,1920x1080,Opera,Russia,Moscow
26714,9172914908724584214.1634328343.1634328343,2135735682.1634328342,2021-10-15,23:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,hJvOVTMdhkqIBqjVrsOL,NaN,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Saint Petersburg
43029,944703987097397663.1624631711.1624631711,219956037.1624631711,2021-06-25,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,DnEUulZAecfGPvdtZBYS,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Moscow
60248,1021042336384541670.1624630248.1624630248,237729944.1624630246,2021-06-25,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,DnEUulZAecfGPvdtZBYS,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Fryazino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852567,9021962630917622540.1622248213.1622248213,2100589366.1622248204,2021-05-29,03:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,360x800,Chrome,Russia,(not set)
1852568,9021962630917622540.1622248280.1622248280,2100589366.1622248204,2021-05-29,03:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,KUROllwAYyecYcjFOgAi,NOBKLgtuvqYWkXQHeYWM,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,360x800,Chrome,Russia,(not set)
1853240,9024892064901457390.1622238706.1622238706,2101271428.1622238702,2021-05-29,00:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Saint Petersburg
1854416,9030278666862206423.1629925388.1629925388,2102525594.1629232599,2021-08-26,00:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,puhZPIYqKXeFPaUviSjo,mobile,None,,AuMdmADEIoPXiWpTsBEj,640x360,Chrome,Russia,Saint Petersburg


In [111]:
df_out_pkl.loc[(df_out_pkl['device_os'].isna()) & (df_out_pkl['device_category'] == 'desktop')]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
130636,1338874692298414890.1630870317.1630870317,311731056.1630870314,2021-09-05,22:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,360x760,Chrome,Russia,Novorossiysk
218862,173540233991179810.1632128548.1632128548,40405484.1632128546,2021-09-20,12:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Nizhny Novgorod
248442,1868400145061735811.1622240641.1622240641,435020808.1622240643,2021-05-29,01:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,360x770,Chrome,Russia,Stavropol
264199,1939665880277002633.1624975755.1624975755,451613655.1624975753,2021-06-29,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,zPJpddwzkFqLMSYgtDqy,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x760,YaBrowser,Russia,Moscow
264201,1939665880277002633.1624975821.1624975821,451613655.1624975753,2021-06-29,17:00:00,3,kjsLglQLzykiRbcDiGcD,cpc,zPJpddwzkFqLMSYgtDqy,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,None,Samsung,AuMdmADEIoPXiWpTsBEj,360x760,YaBrowser,Russia,Moscow
318022,2179795865415458772.1626611671.1626611671,507523274.1626611668,2021-07-18,15:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,Xiaomi,AuMdmADEIoPXiWpTsBEj,360x720,Opera,Russia,Saratov
332623,224549194041055047.1634391882.1634391882,52281933.1634391879,2021-10-16,16:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,RoDitORHdzGfGhNCyEMy,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,Nokia,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow
404371,2564801024506496802.1631881004.1631881004,597164273.1631880994,2021-09-17,15:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,360x592,Chrome,Russia,Saint Petersburg
449802,2763246423595009915.1631569785.1631569785,643368443.1631569787,2021-09-14,00:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,agnCWMgbwJZgTVVsuCLg,eOWmIGTKVDPewucDtZXG,puhZPIYqKXeFPaUviSjo,desktop,None,Huawei,AuMdmADEIoPXiWpTsBEj,534x854,Chrome,Russia,Moscow
524383,309992255509729444.1623691510.1623691510,72175696.1623691428,2021-06-14,20:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,JwYIveaHVpeeRZloQCfF,NaN,puhZPIYqKXeFPaUviSjo,desktop,None,Asus,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Moscow


Оставшиеся устройства - планшеты, смартфоны и ноутбуки. Не имея иных сведений, положим им самые часто встречаемые значения для этих категорий

In [112]:
df_out_pkl.loc[(((df_out_pkl['device_category'] == 'mobile') | (df_out_pkl['device_category'] == 'tablet')) & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_m

In [113]:
df_out_pkl.loc[((df_out_pkl['device_category'] == 'desktop') & (df_out_pkl['device_os'].isna())), 'device_os'] = top_device_os_d

In [114]:
print_missing_values(df_out_pkl)

Процент пропущенных значений:
utm_adcontent               18.046008
utm_campaign                11.807784
device_brand                 6.365396
utm_source                   0.005216
session_id                   0.000000
device_os                    0.000000
geo_country                  0.000000
device_browser               0.000000
device_screen_resolution     0.000000
device_model                 0.000000
utm_keyword                  0.000000
device_category              0.000000
client_id                    0.000000
utm_medium                   0.000000
visit_number                 0.000000
visit_time                   0.000000
visit_date                   0.000000
geo_city                     0.000000
dtype: float64


##### Обработка значений utm_adcontent